In [1]:
!nvidia-smi

Wed Aug 30 09:27:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   60C    P0              31W /  70W |      2MiB / 15360MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.268
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!ls -lah /root/.cache/huggingface/hub/

total 4.0K
drwxr-xr-x 5 root root 149 2023-08-29__13:57:51 .
drwxr-xr-x 3 root root  30 2023-08-16__14:49:17 ..
drwxr-xr-x 2 root root   6 2023-08-29__13:57:51 models--TheBloke--Llama-2-7B-GGML
drwxr-xr-x 6 root root  65 2023-08-21__09:16:52 models--bigscience--bloom-1b7
drwxr-xr-x 6 root root  65 2023-08-16__17:33:10 models--meta-llama--Llama-2-7b-chat-hf
-rw-r--r-- 1 root root   1 2023-07-26__16:10:47 version.txt


In [4]:
# import sys,os,os.path
# os.environ['HTTP_PROXY']="http://127.0.0.1:8098"
# os.environ['HTTPS_PROXY']="http://127.0.0.1:8098"

In [5]:
import torch
torch.cuda.device_count()

1

In [6]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [7]:
# https://huggingface.co/bigscience/bloom-1b7

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
from transformers import BloomTokenizerFast

In [10]:
# tokenizer = BloomTokenizerFast.from_pretrained(
#     "bigscience/bloom-1b7",
#     local_files_only=True
# )

In [11]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name,
                                          local_files_only=True
                                          # use_auth_token=True,
                                         )

In [13]:
tokenizer.eos_token_id

2

In [14]:
tokenizer.pad_token_id

In [15]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_bos_token',
 '_add_eos_token',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_build_conversation_input_ids',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in

In [16]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             local_files_only=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             # temperature=0.1, # must be strictly positive float
                                             do_sample=True,
                                             # use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

2023-08-30 09:27:25.547011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 09:27:26.452656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
!nvidia-smi

Wed Aug 30 09:27:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   60C    P0              31W /  70W |  13095MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [18]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [19]:
!nvidia-smi

Wed Aug 30 09:27:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:AF:00.0 Off |                    0 |
| N/A   60C    P0              31W /  70W |  13095MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
from langchain.vectorstores import Chroma
persist_directory = 'db'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [21]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [22]:
# https://python.langchain.com/docs/modules/model_io/models/chat/prompts

In [23]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # return_full_text=True,
                # torch_dtype=torch.bfloat16,
                # device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [24]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [25]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

In [26]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [27]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [28]:
qa({"query": question1})

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 14.58 GiB total capacity; 14.36 GiB already allocated; 45.56 MiB free; 14.36 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# 1

In [ ]:
question1 = "How can I navigate to the RMA request page in the Wareconn Customer Portal?"

In [ ]:
%%time
qa({"query": question1})

# 2

In [ ]:
question2 = "What does the Reminder field indicate when submitting an RMA request?"

In [ ]:
%%time
qa({"query": question2})

In [ ]:
question3 = "What information should be filled in the Customer No. field when submitting an RMA request?"

In [ ]:
%%time
qa({"query": question3})

In [ ]:
question4 = "What steps should be followed when filling in the request information for an RMA?"

In [ ]:
%%time
qa({"query": question4})

In [ ]:
question5 = "Can this document be used for purposes other than introducing Wareconn functions and procedures?"

In [ ]:
%%time
qa({"query": question5})